# Yelp Navigator - Pipeline Chaining to Summary Reports

This notebook demonstrates **2 workflow patterns** for generating business reports:

1. **Pipeline 1 + 2** (Basic info + website details)
2. **Pipeline 1 + 3** (Basic info + reviews/sentiment)

## Prerequisites

- **Environment variables** in `../.env` (including `RAPID_API_KEY` and `OPENAI_API_KEY`)
- **Hayhooks server running**: `sh start_hayhooks.sh`
- Server at `http://localhost:1416`

## Setup and Imports

In [1]:
import requests
import json
from pprint import pprint
from typing import Dict, Any

# Base URL for Hayhooks server
BASE_URL = "http://localhost:1416"

# Helper function to print JSON nicely
def print_json(data, max_items=3):
    """Print JSON data in a readable format"""
    print(json.dumps(data, indent=2)[:2000])  # Limit output length

## Test Server Connection

In [2]:
# Check if Hayhooks server is running
try:
    response = requests.get(f"{BASE_URL}/status")
    print("Hayhooks server is running!")
    print(f"Status: {response.status_code}")
except requests.exceptions.ConnectionError:
    print("Cannot connect to Hayhooks server")
    print("Please start the server with: hayhooks run --pipelines-dir pipelines")

Hayhooks server is running!
Status: 200


---

# Step 1: Run Pipeline 1 (Business Search)

**Purpose**: Get basic business information from Yelp search

This pipeline extracts location and query details, searches Yelp, and returns business data.

In [3]:
# Workflow 1: Search + Basic Report
query = "Italian restaurants in San Francisco and Vancouver"

print("Step 1: Running Pipeline 1 (Business Search)...")
response1 = requests.post(f"{BASE_URL}/business_search/run", json={"query": query})
pipeline1_output = response1.json()

Step 1: Running Pipeline 1 (Business Search)...


In [4]:
pipeline1_output

{'result': {'query': 'Italian restaurants in San Francisco and Vancouver',
  'extracted_location': ['San Francisco', 'Vancouver'],
  'extracted_keywords': [['restaurants', 'Italian'],
   ['restaurants', 'Italian']],
  'extracted_locations': ['San Francisco', 'Vancouver'],
  'extracted_keywords_list': [['restaurants', 'Italian'],
   ['restaurants', 'Italian']],
  'original_queries': ['restaurants, Italian in San Francisco',
   'restaurants, Italian in Vancouver'],
  'search_params': {'searches': [{'location': 'San Francisco',
     'query': 'restaurants Italian',
     'original_query': 'restaurants, Italian in San Francisco',
     'result_count': 240},
    {'location': 'Vancouver',
     'query': 'restaurants Italian',
     'original_query': 'restaurants, Italian in Vancouver',
     'result_count': 240}],
   'combined_query': 'restaurants, Italian in San Francisco AND restaurants, Italian in Vancouver'},
  'result_count': 20,
  'is_multi_query': True,
  'total_searches': 2,
  'businesses'

---

# Workflow 1: Pipeline 1 + 2 

**Use case**: Business overview + website details & offerings

**What it does**: Takes Pipeline 1 output and enriches it with website information

**Speed**: Medium (~10-15 seconds)

In [5]:

print("Step 2: Running Pipeline 2 (Website Details)...")
response2 = requests.post(
        f"{BASE_URL}/business_details/run",
        json={"pipeline1_output": pipeline1_output}
    )
pipeline2_output = response2.json()


Step 2: Running Pipeline 2 (Website Details)...


In [6]:
pipeline2_output


{'result': {'document_count': 17,
  'documents': [{'content': 'SF Beer Week Click to share on X (Opens in new window) X Click to share on Facebook (Opens in new window) Facebook Like this:Like Loading...',
    'metadata': {'content_type': 'text/html',
     'url': 'http://Www.bellatrattoriasf.com',
     'business_id': 'msT3LrLB4fhN04HYHuFsew',
     'business_name': 'Bella Trattoria',
     'business_alias': 'bella-trattoria-san-francisco',
     'price_range': '$$',
     'latitude': 37.78136,
     'longitude': -122.46092108,
     'rating': 4.3,
     'review_count': 2064,
     'phone': '(415) 221-0305',
     'categories': ['Italian', 'Bars', 'Pasta Shops'],
     'website': 'http://Www.bellatrattoriasf.com',
     'images': ['https://s3-media0.fl.yelpcdn.com/bphoto/lDIjLb1APIFuSSltL1L4vw/348s.jpg']}},
   {'content': 'top of page1/12"The Best Damn Cioppino in San Francisco!"Located in the heart of North Beach, San Francisco, Sotto Mare Restaurant provides a delicious and authentic Italian Nor

---

# Workflow 2: Pipeline 1 + 3 

**Use case**: Business overview + customer reviews & sentiment

**What it does**: Takes Pipeline 1 output and enriches it with reviews and sentiment analysis

**Speed**: Slower (~30-60 seconds)

In [7]:
# Workflow 3: Search + Reviews/Sentiment + Report
print("Step 3: Running Pipeline 3 (Business Sentiment)...")
response3 = requests.post(
        f"{BASE_URL}/business_sentiment/run",
        json={"pipeline1_output": pipeline1_output},
        timeout=120
    )
pipeline3_output = response3.json()


Step 3: Running Pipeline 3 (Business Sentiment)...


In [8]:
pipeline3_output

{'result': {'business_count': 20,
  'business_ids_processed': ['msT3LrLB4fhN04HYHuFsew',
   '8dUaybEPHsZMgr1iKgqgMQ',
   'B09WOy0W83Od-Xw4xEXxog',
   'QueFVMcMlT-6aZFv2M47mg',
   'oK6DQM2ztdNwMAyJqGWZpg',
   'FvPRM23d7NAOdyS_OX0MpQ',
   'K9XVDlPNhrrSVEJN7uWqJQ',
   'o43B4DnnQbvkdDK6AVafQg',
   'zREcSPXvizDj-79lSw1MSQ',
   '6e7gMgVJRQ5rtkIbTF6TLQ',
   'htvl5L_V-zKN0UvUgP60PQ',
   'V-kQkPZzlOXIqk3rSqkikg',
   'DkKH_q0_vdWSWZwFhPFLZw',
   'g-WSOodoG-L81DFYWM0FMQ',
   'wHTXCWVsx2PYPT21JV-HeQ',
   'k7-oClu97qColM6BcCaQMw',
   'xWooPFPiXv8PZ2F6IBKjnA',
   'GepW_CkeEoPbg-0fLb5FGw',
   'bgQRJLO8QFYDpCfkeOhjVA',
   'gNCSbTm3unsdztncbgSv5A'],
  'total_reviews_analyzed': 200,
  'businesses': [{'business_id': 'msT3LrLB4fhN04HYHuFsew',
    'total_reviews': 10,
    'sentiment_distribution': {'positive': 8, 'neutral': 1, 'negative': 1},
    'sentiment_percentages': {'positive': 80.0,
     'neutral': 10.0,
     'negative': 10.0},
    'overall_sentiment': 'positive',
    'highest_rated_reviews': [{'rat